In [ ]:
!nvidia-smi

Thu Jun  2 15:45:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8701796529032470077
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 16154099712
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 316733871397505733
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
 xla_global_id: 416903419]

#ALBERT

In [ ]:
!pip install transformers
import transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install SentencePiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from tqdm.notebook import tqdm
from transformers import  AlbertTokenizer, AlbertModel
from torch.utils.data import TensorDataset

from transformers import AlbertTokenizer, AlbertForSequenceClassification
import pandas as pd

In [ ]:
!gdown --id 15UFD8alS79ri-gBnaWTKy0iXSES2mJMY
#gets labeled data

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=15UFD8alS79ri-gBnaWTKy0iXSES2mJMY
To: /content/earning_annotation.csv
100% 98.8k/98.8k [00:00<00:00, 85.6MB/s]


In [ ]:
import pandas as pd, csv, numpy as np
df=pd.read_csv('earning_annotation.csv')

In [ ]:
N = 7
# Select first N columns
df = df.iloc[: , :N]

In [ ]:
df.head(20)

,id,text,gone,gtwo,machine,human,label
0,MSFT_q4_2018.txt,And our commercial cloud business surpassed mo...,['cloud'],['Cloud computing'],Past,Past,0
1,MSFT_q4_2018.txt,I shared our vision for the intelligent cloud ...,['cloud'],['Cloud computing'],Past,Past,0
2,MSFT_q4_2018.txt,We reorganized our engineering teams to break ...,['AI'],['AI'],Past,Past,0
3,MSFT_q4_2018.txt,"We reoriented our sales and marketing teams, a...","['transformation', 'digital transformation']",['Practices'],Past,Past,0
4,MSFT_q4_2018.txt,"And, most importantly, we drove innovation to ...","['cloud', 'the cloud']",['Cloud computing'],Past,Past,0
5,MSFT_q4_2018.txt,Microsoft 365 is now a multi-billion-dollar bu...,"['cloud', 'the cloud']",['Cloud computing'],Pres,Pres,1
6,MSFT_q4_2018.txt,"Across Microsoft 365, we are helping people be...",['AI'],['AI'],Pres,Pres,1
7,MSFT_q4_2018.txt,Outlook mobile is being used on more than 100 ...,['mobile'],['Mobile'],Pres,Pres,1
8,MSFT_q4_2018.txt,This strong engagement is driven by quality of...,['mobile'],['Mobile'],Pres,Pres,1
9,MSFT_q4_2018.txt,We will continue to invest to make LinkedIn th...,['platform'],['Digital Product'],Fut,Pres,1


In [ ]:
df=df.head(401)

In [ ]:
df

,id,text,gone,gtwo,machine,human,label
0,MSFT_q4_2018.txt,And our commercial cloud business surpassed mo...,['cloud'],['Cloud computing'],Past,Past,0
1,MSFT_q4_2018.txt,I shared our vision for the intelligent cloud ...,['cloud'],['Cloud computing'],Past,Past,0
2,MSFT_q4_2018.txt,We reorganized our engineering teams to break ...,['AI'],['AI'],Past,Past,0
3,MSFT_q4_2018.txt,"We reoriented our sales and marketing teams, a...","['transformation', 'digital transformation']",['Practices'],Past,Past,0
4,MSFT_q4_2018.txt,"And, most importantly, we drove innovation to ...","['cloud', 'the cloud']",['Cloud computing'],Past,Past,0
...,...,...,...,...,...,...,...
396,IBM_q1_2018.txt,We’ve grown to over 50 active blockchain netwo...,"['platform', 'blockchain']","['Blockchain', 'Digital Product']",Pres,Pres,1
397,IBM_q1_2018.txt,"And last month, we announced a beta version of...","['platform', 'blockchain']","['Blockchain', 'Digital Product']",Past,Past,0
398,IBM_q1_2018.txt,Our strategic imperatives revenue reflects the...,['cloud'],['Cloud computing'],Pres,Pres,1
399,IBM_q1_2018.txt,Revenue growth was led by strong double-digit ...,"['transformation', 'digital transformation', '...","['Practices', 'Enablers']",Past,Past,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.25, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [ ]:
X_val

array([242, 214,  58, 300, 273,   7, 337, 234, 180, 146,  95, 235, 198,
       382,  97, 363,   1, 262, 113, 367, 173, 292, 323, 248, 326, 280,
        57, 134, 139, 112, 333, 116, 398, 334,  59, 176, 167,  67, 159,
        77, 299, 205,  37, 165,  29, 160, 394, 226,  94, 344, 396,  60,
       381, 339, 314, 389, 245, 357,  61, 128, 335, 137, 378, 105, 120,
        79, 210, 261, 233, 331, 296, 373, 174, 354,  22, 204,  85, 163,
       279, 127, 237,  84, 284, 241, 133, 399, 308,  38,  20, 229, 240,
       306, 254, 208, 211, 223,  75, 293, 386, 349, 191])

In [ ]:
y_train

array([0, 2, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 2, 1, 0,
       1, 1, 1, 0, 1, 2, 2, 1, 2, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 2, 1, 0, 1, 1, 0, 2, 1, 2, 0, 1, 1, 0, 0, 1, 0, 0, 0, 2, 0,
       2, 0, 1, 2, 0, 0, 2, 2, 0, 1, 0, 0, 2, 1, 0, 0, 2, 2, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 2, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 2, 2,
       1, 0, 2, 1, 0, 2, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 2, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 1, 2, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 2, 1,
       1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 2, 2, 1, 2, 1, 2, 0,
       1, 1, 0, 0, 2, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 0, 2, 2, 1, 1,
       0, 1, 0, 2, 1, 2, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 2, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 0, 1, 2, 1, 0, 2, 1, 0,
       0, 1, 0, 0, 2, 0, 0, 2, 1, 1, 0, 1, 1, 1])

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['label', 'data_type']).count()

id  text  gone  gtwo  machine  human
label data_type                                       
0     train      106   106   106   106      106    106
      val         36    36    36    36       36     36
1     train      147   147   147   147      144    147
      val         49    49    49    49       49     49
2     train       47    47    47    47       46     47
      val         16    16    16    16       16     16

In [ ]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

Downloading:   0%|          | 0.00/742k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train), len(dataset_val)

(300, 101)

In [ ]:
from transformers import AlbertTokenizer, AlbertForSequenceClassification

In [ ]:
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2',
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/45.2M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.dense.bias', 'predictions.dense.weight', 'predictions.LayerNorm.bias', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You sho

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
epochs = 6

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    #label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        # print(f'Class: {label_dict_inverse[label]}')
        print(f'Class: {label}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

def predict(dataloader_test):

    model.eval()
    
    predictions = []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]
                  }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        # loss = outputs[0]
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
        
    predictions = np.concatenate(predictions, axis=0)            
    return predictions

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    # torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.8835835164785385
Validation loss: 0.8371686427032247
F1 Score (Weighted): 0.5868199723198126


Epoch 2:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.8558099594712257
Validation loss: 0.9805184181998757
F1 Score (Weighted): 0.3690278184271203


...


Epoch 4
Training loss: 0.46015548422932623
Validation loss: 0.5789895498139017
F1 Score (Weighted): 0.7505922726560387


Epoch 5:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.29617725905030967
Validation loss: 0.562656505855129
F1 Score (Weighted): 0.8372781046220564


Epoch 6:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 6
Training loss: 0.22998764291405677
Validation loss: 0.5483184941660832
F1 Score (Weighted): 0.8177363407982589


In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: 0
Accuracy: 28/36

Class: 1
Accuracy: 46/49

Class: 2
Accuracy: 9/16



In [ ]:
%reset -f

#BERT- CASED

In [ ]:
!gdown --id 15UFD8alS79ri-gBnaWTKy0iXSES2mJMY
#gets labeled data

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=15UFD8alS79ri-gBnaWTKy0iXSES2mJMY
To: /content/earning_annotation.csv
100% 98.8k/98.8k [00:00<00:00, 89.8MB/s]


In [ ]:
import pandas as pd, csv, numpy as np
df=pd.read_csv('earning_annotation.csv')

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer


from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
import pandas as pd

In [ ]:
N = 7
# Select first N columns
df = df.iloc[: , :N]

In [ ]:
df.head(20)

,id,text,gone,gtwo,machine,human,label
0,MSFT_q4_2018.txt,And our commercial cloud business surpassed mo...,['cloud'],['Cloud computing'],Past,Past,0
1,MSFT_q4_2018.txt,I shared our vision for the intelligent cloud ...,['cloud'],['Cloud computing'],Past,Past,0
2,MSFT_q4_2018.txt,We reorganized our engineering teams to break ...,['AI'],['AI'],Past,Past,0
3,MSFT_q4_2018.txt,"We reoriented our sales and marketing teams, a...","['transformation', 'digital transformation']",['Practices'],Past,Past,0
4,MSFT_q4_2018.txt,"And, most importantly, we drove innovation to ...","['cloud', 'the cloud']",['Cloud computing'],Past,Past,0
5,MSFT_q4_2018.txt,Microsoft 365 is now a multi-billion-dollar bu...,"['cloud', 'the cloud']",['Cloud computing'],Pres,Pres,1
6,MSFT_q4_2018.txt,"Across Microsoft 365, we are helping people be...",['AI'],['AI'],Pres,Pres,1
7,MSFT_q4_2018.txt,Outlook mobile is being used on more than 100 ...,['mobile'],['Mobile'],Pres,Pres,1
8,MSFT_q4_2018.txt,This strong engagement is driven by quality of...,['mobile'],['Mobile'],Pres,Pres,1
9,MSFT_q4_2018.txt,We will continue to invest to make LinkedIn th...,['platform'],['Digital Product'],Fut,Pres,1


In [ ]:
df=df.head(401)

In [ ]:
df

,id,text,gone,gtwo,machine,human,label
0,MSFT_q4_2018.txt,And our commercial cloud business surpassed mo...,['cloud'],['Cloud computing'],Past,Past,0
1,MSFT_q4_2018.txt,I shared our vision for the intelligent cloud ...,['cloud'],['Cloud computing'],Past,Past,0
2,MSFT_q4_2018.txt,We reorganized our engineering teams to break ...,['AI'],['AI'],Past,Past,0
3,MSFT_q4_2018.txt,"We reoriented our sales and marketing teams, a...","['transformation', 'digital transformation']",['Practices'],Past,Past,0
4,MSFT_q4_2018.txt,"And, most importantly, we drove innovation to ...","['cloud', 'the cloud']",['Cloud computing'],Past,Past,0
...,...,...,...,...,...,...,...
396,IBM_q1_2018.txt,We’ve grown to over 50 active blockchain netwo...,"['platform', 'blockchain']","['Blockchain', 'Digital Product']",Pres,Pres,1
397,IBM_q1_2018.txt,"And last month, we announced a beta version of...","['platform', 'blockchain']","['Blockchain', 'Digital Product']",Past,Past,0
398,IBM_q1_2018.txt,Our strategic imperatives revenue reflects the...,['cloud'],['Cloud computing'],Pres,Pres,1
399,IBM_q1_2018.txt,Revenue growth was led by strong double-digit ...,"['transformation', 'digital transformation', '...","['Practices', 'Enablers']",Past,Past,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.25, 
                                                  random_state=42, 
                                                  stratify=df.label.values)


In [ ]:
X_val

array([242, 214,  58, 300, 273,   7, 337, 234, 180, 146,  95, 235, 198,
       382,  97, 363,   1, 262, 113, 367, 173, 292, 323, 248, 326, 280,
        57, 134, 139, 112, 333, 116, 398, 334,  59, 176, 167,  67, 159,
        77, 299, 205,  37, 165,  29, 160, 394, 226,  94, 344, 396,  60,
       381, 339, 314, 389, 245, 357,  61, 128, 335, 137, 378, 105, 120,
        79, 210, 261, 233, 331, 296, 373, 174, 354,  22, 204,  85, 163,
       279, 127, 237,  84, 284, 241, 133, 399, 308,  38,  20, 229, 240,
       306, 254, 208, 211, 223,  75, 293, 386, 349, 191])

In [ ]:
y_train

array([0, 2, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 2, 1, 0,
       1, 1, 1, 0, 1, 2, 2, 1, 2, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 2, 1, 0, 1, 1, 0, 2, 1, 2, 0, 1, 1, 0, 0, 1, 0, 0, 0, 2, 0,
       2, 0, 1, 2, 0, 0, 2, 2, 0, 1, 0, 0, 2, 1, 0, 0, 2, 2, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 2, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 2, 2,
       1, 0, 2, 1, 0, 2, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 2, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 1, 2, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 2, 1,
       1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 2, 2, 1, 2, 1, 2, 0,
       1, 1, 0, 0, 2, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 0, 2, 2, 1, 1,
       0, 1, 0, 2, 1, 2, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 2, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 0, 1, 2, 1, 0, 2, 1, 0,
       0, 1, 0, 0, 2, 0, 0, 2, 1, 1, 0, 1, 1, 1])

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['label', 'data_type']).count()

id  text  gone  gtwo  machine  human
label data_type                                       
0     train      106   106   106   106      106    106
      val         36    36    36    36       36     36
1     train      147   147   147   147      144    147
      val         49    49    49    49       49     49
2     train       47    47    47    47       46     47
      val         16    16    16    16       16     16

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train), len(dataset_val)

(300, 101)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-cased",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5,
                  eps=1e-8)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
epochs = 6

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    #label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        # print(f'Class: {label_dict_inverse[label]}')
        print(f'Class: {label}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

def predict(dataloader_test):

    model.eval()
    
    predictions = []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]
                  }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        # loss = outputs[0]
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
        
    predictions = np.concatenate(predictions, axis=0)            
    return predictions

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    # torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.8823474758863449
Validation loss: 0.5335021913051605
F1 Score (Weighted): 0.8655110402144911


Epoch 2:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.3041138787195086
Validation loss: 0.44452706927104907
F1 Score (Weighted): 0.9004727499777004


...


Epoch 4
Training loss: 0.06705886931158603
Validation loss: 0.5829160537926809
F1 Score (Weighted): 0.9106075565618071


Epoch 5:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.04817854217020795
Validation loss: 0.594995814084541
F1 Score (Weighted): 0.9106075565618071


Epoch 6:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 6
Training loss: 0.03188981787883677
Validation loss: 0.6015527820289956
F1 Score (Weighted): 0.9106075565618071


In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: 0
Accuracy: 34/36

Class: 1
Accuracy: 45/49

Class: 2
Accuracy: 13/16



In [ ]:
!nvidia-smi

Thu Jun  2 15:50:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    53W / 250W |   4733MiB / 16280MiB |     48%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%reset -f

#BERT- UNCASED

In [ ]:
!gdown --id 15UFD8alS79ri-gBnaWTKy0iXSES2mJMY
#gets labeled data

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=15UFD8alS79ri-gBnaWTKy0iXSES2mJMY
To: /content/earning_annotation.csv
100% 98.8k/98.8k [00:00<00:00, 93.4MB/s]


In [ ]:
import pandas as pd, csv, numpy as np
df=pd.read_csv('earning_annotation.csv')

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer


from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
import pandas as pd

In [ ]:
N = 7
# Select first N columns
df = df.iloc[: , :N]

In [ ]:
df.head(20)

,id,text,gone,gtwo,machine,human,label
0,MSFT_q4_2018.txt,And our commercial cloud business surpassed mo...,['cloud'],['Cloud computing'],Past,Past,0
1,MSFT_q4_2018.txt,I shared our vision for the intelligent cloud ...,['cloud'],['Cloud computing'],Past,Past,0
2,MSFT_q4_2018.txt,We reorganized our engineering teams to break ...,['AI'],['AI'],Past,Past,0
3,MSFT_q4_2018.txt,"We reoriented our sales and marketing teams, a...","['transformation', 'digital transformation']",['Practices'],Past,Past,0
4,MSFT_q4_2018.txt,"And, most importantly, we drove innovation to ...","['cloud', 'the cloud']",['Cloud computing'],Past,Past,0
5,MSFT_q4_2018.txt,Microsoft 365 is now a multi-billion-dollar bu...,"['cloud', 'the cloud']",['Cloud computing'],Pres,Pres,1
6,MSFT_q4_2018.txt,"Across Microsoft 365, we are helping people be...",['AI'],['AI'],Pres,Pres,1
7,MSFT_q4_2018.txt,Outlook mobile is being used on more than 100 ...,['mobile'],['Mobile'],Pres,Pres,1
8,MSFT_q4_2018.txt,This strong engagement is driven by quality of...,['mobile'],['Mobile'],Pres,Pres,1
9,MSFT_q4_2018.txt,We will continue to invest to make LinkedIn th...,['platform'],['Digital Product'],Fut,Pres,1


In [ ]:
df=df.head(401)

In [ ]:
df

,id,text,gone,gtwo,machine,human,label
0,MSFT_q4_2018.txt,And our commercial cloud business surpassed mo...,['cloud'],['Cloud computing'],Past,Past,0
1,MSFT_q4_2018.txt,I shared our vision for the intelligent cloud ...,['cloud'],['Cloud computing'],Past,Past,0
2,MSFT_q4_2018.txt,We reorganized our engineering teams to break ...,['AI'],['AI'],Past,Past,0
3,MSFT_q4_2018.txt,"We reoriented our sales and marketing teams, a...","['transformation', 'digital transformation']",['Practices'],Past,Past,0
4,MSFT_q4_2018.txt,"And, most importantly, we drove innovation to ...","['cloud', 'the cloud']",['Cloud computing'],Past,Past,0
...,...,...,...,...,...,...,...
396,IBM_q1_2018.txt,We’ve grown to over 50 active blockchain netwo...,"['platform', 'blockchain']","['Blockchain', 'Digital Product']",Pres,Pres,1
397,IBM_q1_2018.txt,"And last month, we announced a beta version of...","['platform', 'blockchain']","['Blockchain', 'Digital Product']",Past,Past,0
398,IBM_q1_2018.txt,Our strategic imperatives revenue reflects the...,['cloud'],['Cloud computing'],Pres,Pres,1
399,IBM_q1_2018.txt,Revenue growth was led by strong double-digit ...,"['transformation', 'digital transformation', '...","['Practices', 'Enablers']",Past,Past,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.25, 
                                                  random_state=42, 
                                                  stratify=df.label.values)


In [ ]:
X_val

array([242, 214,  58, 300, 273,   7, 337, 234, 180, 146,  95, 235, 198,
       382,  97, 363,   1, 262, 113, 367, 173, 292, 323, 248, 326, 280,
        57, 134, 139, 112, 333, 116, 398, 334,  59, 176, 167,  67, 159,
        77, 299, 205,  37, 165,  29, 160, 394, 226,  94, 344, 396,  60,
       381, 339, 314, 389, 245, 357,  61, 128, 335, 137, 378, 105, 120,
        79, 210, 261, 233, 331, 296, 373, 174, 354,  22, 204,  85, 163,
       279, 127, 237,  84, 284, 241, 133, 399, 308,  38,  20, 229, 240,
       306, 254, 208, 211, 223,  75, 293, 386, 349, 191])

In [ ]:
y_train

array([0, 2, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 2, 1, 0,
       1, 1, 1, 0, 1, 2, 2, 1, 2, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 2, 1, 0, 1, 1, 0, 2, 1, 2, 0, 1, 1, 0, 0, 1, 0, 0, 0, 2, 0,
       2, 0, 1, 2, 0, 0, 2, 2, 0, 1, 0, 0, 2, 1, 0, 0, 2, 2, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 2, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 2, 2,
       1, 0, 2, 1, 0, 2, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 2, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 1, 2, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 2, 1,
       1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 2, 2, 1, 2, 1, 2, 0,
       1, 1, 0, 0, 2, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 0, 2, 2, 1, 1,
       0, 1, 0, 2, 1, 2, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 2, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 0, 1, 2, 1, 0, 2, 1, 0,
       0, 1, 0, 0, 2, 0, 0, 2, 1, 1, 0, 1, 1, 1])

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['label', 'data_type']).count()

id  text  gone  gtwo  machine  human
label data_type                                       
0     train      106   106   106   106      106    106
      val         36    36    36    36       36     36
1     train      147   147   147   147      144    147
      val         49    49    49    49       49     49
2     train       47    47    47    47       46     47
      val         16    16    16    16       16     16

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train), len(dataset_val)

(300, 101)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
epochs = 6

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    #label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        # print(f'Class: {label_dict_inverse[label]}')
        print(f'Class: {label}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

def predict(dataloader_test):

    model.eval()
    
    predictions = []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]
                  }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        # loss = outputs[0]
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
        
    predictions = np.concatenate(predictions, axis=0)            
    return predictions

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    # torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.9507094737887383
Validation loss: 0.7777528420967215
F1 Score (Weighted): 0.6858762286174038


Epoch 2:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.47859598748385906
Validation loss: 0.432625782182988
F1 Score (Weighted): 0.8809383641066808


...


Epoch 4
Training loss: 0.10685226818546653
Validation loss: 0.3474435494144392
F1 Score (Weighted): 0.930186019626203


Epoch 5:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.06428639952093362
Validation loss: 0.3733193300773992
F1 Score (Weighted): 0.930186019626203


Epoch 6:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 6
Training loss: 0.044637399767525496
Validation loss: 0.37854105205384686
F1 Score (Weighted): 0.930186019626203


In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: 0
Accuracy: 34/36

Class: 1
Accuracy: 47/49

Class: 2
Accuracy: 13/16



In [ ]:
%reset -f

#RoBERTa

In [ ]:
!pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown --id 15UFD8alS79ri-gBnaWTKy0iXSES2mJMY
#gets labeled data

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=15UFD8alS79ri-gBnaWTKy0iXSES2mJMY
To: /content/earning_annotation.csv
100% 98.8k/98.8k [00:00<00:00, 58.7MB/s]


In [ ]:
import pandas as pd, csv, numpy as np
df=pd.read_csv('earning_annotation.csv')

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from tqdm.notebook import tqdm
from transformers import RobertaConfig, RobertaModel
from torch.utils.data import TensorDataset

import pandas as pd

In [ ]:
N = 7
# Select first N columns
df = df.iloc[: , :N]

In [ ]:
df.head(20)

,id,text,gone,gtwo,machine,human,label
0,MSFT_q4_2018.txt,And our commercial cloud business surpassed mo...,['cloud'],['Cloud computing'],Past,Past,0
1,MSFT_q4_2018.txt,I shared our vision for the intelligent cloud ...,['cloud'],['Cloud computing'],Past,Past,0
2,MSFT_q4_2018.txt,We reorganized our engineering teams to break ...,['AI'],['AI'],Past,Past,0
3,MSFT_q4_2018.txt,"We reoriented our sales and marketing teams, a...","['transformation', 'digital transformation']",['Practices'],Past,Past,0
4,MSFT_q4_2018.txt,"And, most importantly, we drove innovation to ...","['cloud', 'the cloud']",['Cloud computing'],Past,Past,0
5,MSFT_q4_2018.txt,Microsoft 365 is now a multi-billion-dollar bu...,"['cloud', 'the cloud']",['Cloud computing'],Pres,Pres,1
6,MSFT_q4_2018.txt,"Across Microsoft 365, we are helping people be...",['AI'],['AI'],Pres,Pres,1
7,MSFT_q4_2018.txt,Outlook mobile is being used on more than 100 ...,['mobile'],['Mobile'],Pres,Pres,1
8,MSFT_q4_2018.txt,This strong engagement is driven by quality of...,['mobile'],['Mobile'],Pres,Pres,1
9,MSFT_q4_2018.txt,We will continue to invest to make LinkedIn th...,['platform'],['Digital Product'],Fut,Pres,1


In [ ]:
df=df.head(401)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.25, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [ ]:
X_val

array([242, 214,  58, 300, 273,   7, 337, 234, 180, 146,  95, 235, 198,
       382,  97, 363,   1, 262, 113, 367, 173, 292, 323, 248, 326, 280,
        57, 134, 139, 112, 333, 116, 398, 334,  59, 176, 167,  67, 159,
        77, 299, 205,  37, 165,  29, 160, 394, 226,  94, 344, 396,  60,
       381, 339, 314, 389, 245, 357,  61, 128, 335, 137, 378, 105, 120,
        79, 210, 261, 233, 331, 296, 373, 174, 354,  22, 204,  85, 163,
       279, 127, 237,  84, 284, 241, 133, 399, 308,  38,  20, 229, 240,
       306, 254, 208, 211, 223,  75, 293, 386, 349, 191])

In [ ]:
y_train

array([0, 2, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 2, 1, 0,
       1, 1, 1, 0, 1, 2, 2, 1, 2, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 2, 1, 0, 1, 1, 0, 2, 1, 2, 0, 1, 1, 0, 0, 1, 0, 0, 0, 2, 0,
       2, 0, 1, 2, 0, 0, 2, 2, 0, 1, 0, 0, 2, 1, 0, 0, 2, 2, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 2, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 2, 2,
       1, 0, 2, 1, 0, 2, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 2, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 1, 2, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 2, 1,
       1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 2, 2, 1, 2, 1, 2, 0,
       1, 1, 0, 0, 2, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 0, 2, 2, 1, 1,
       0, 1, 0, 2, 1, 2, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 2, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 0, 1, 2, 1, 0, 2, 1, 0,
       0, 1, 0, 0, 2, 0, 0, 2, 1, 1, 0, 1, 1, 1])

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['label', 'data_type']).count()

id  text  gone  gtwo  machine  human
label data_type                                       
0     train      106   106   106   106      106    106
      val         36    36    36    36       36     36
1     train      147   147   147   147      144    147
      val         49    49    49    49       49     49
2     train       47    47    47    47       46     47
      val         16    16    16    16       16     16

In [ ]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train), len(dataset_val) 

(300, 101)

In [ ]:

from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
from transformers import RobertaTokenizer, RobertaForSequenceClassification

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
model =  RobertaForSequenceClassification.from_pretrained('roberta-base',
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
epochs = 6

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    #label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        # print(f'Class: {label_dict_inverse[label]}')
        print(f'Class: {label}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

def predict(dataloader_test):

    model.eval()
    
    predictions = []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]
                  }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        # loss = outputs[0]
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
        
    predictions = np.concatenate(predictions, axis=0)            
    return predictions

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    # torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.0369037023186685
Validation loss: 0.8573363844086143
F1 Score (Weighted): 0.6348051021318348


Epoch 2:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.6328782145678997
Validation loss: 0.5306535790510037
F1 Score (Weighted): 0.841004370707341


...


Epoch 4
Training loss: 0.16881090636597945
Validation loss: 0.6800590833128595
F1 Score (Weighted): 0.8606077159440081


Epoch 5:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.11755099600413814
Validation loss: 0.7657526285175289
F1 Score (Weighted): 0.8606077159440081


Epoch 6:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 6
Training loss: 0.08058413553517312
Validation loss: 0.7871977308634942
F1 Score (Weighted): 0.8497741202691697


In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: 0
Accuracy: 33/36

Class: 1
Accuracy: 42/49

Class: 2
Accuracy: 11/16



In [ ]:
%reset -f

#FinBERT

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown --id 15UFD8alS79ri-gBnaWTKy0iXSES2mJMY
#gets labeled data

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=15UFD8alS79ri-gBnaWTKy0iXSES2mJMY
To: /content/earning_annotation.csv
100% 98.8k/98.8k [00:00<00:00, 93.0MB/s]


In [ ]:
import pandas as pd, csv, numpy as np
df=pd.read_csv('earning_annotation.csv')

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import BertTokenizer, BertModel, AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup

In [ ]:
import torch
from tqdm.notebook import tqdm
# Import generic wrappers
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification




# Define the model repo
model_name = "ProsusAI/finbert" 
# Download pytorch model
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

from torch.utils.data import TensorDataset
import pandas as pd

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at ProsusAI/finbert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

In [ ]:
N = 7
# Select first N columns
df = df.iloc[: , :N]

In [ ]:
df.head(20)

,id,text,gone,gtwo,machine,human,label
0,MSFT_q4_2018.txt,And our commercial cloud business surpassed mo...,['cloud'],['Cloud computing'],Past,Past,0
1,MSFT_q4_2018.txt,I shared our vision for the intelligent cloud ...,['cloud'],['Cloud computing'],Past,Past,0
2,MSFT_q4_2018.txt,We reorganized our engineering teams to break ...,['AI'],['AI'],Past,Past,0
3,MSFT_q4_2018.txt,"We reoriented our sales and marketing teams, a...","['transformation', 'digital transformation']",['Practices'],Past,Past,0
4,MSFT_q4_2018.txt,"And, most importantly, we drove innovation to ...","['cloud', 'the cloud']",['Cloud computing'],Past,Past,0
5,MSFT_q4_2018.txt,Microsoft 365 is now a multi-billion-dollar bu...,"['cloud', 'the cloud']",['Cloud computing'],Pres,Pres,1
6,MSFT_q4_2018.txt,"Across Microsoft 365, we are helping people be...",['AI'],['AI'],Pres,Pres,1
7,MSFT_q4_2018.txt,Outlook mobile is being used on more than 100 ...,['mobile'],['Mobile'],Pres,Pres,1
8,MSFT_q4_2018.txt,This strong engagement is driven by quality of...,['mobile'],['Mobile'],Pres,Pres,1
9,MSFT_q4_2018.txt,We will continue to invest to make LinkedIn th...,['platform'],['Digital Product'],Fut,Pres,1


In [ ]:
df=df.head(401)

In [ ]:
df

,id,text,gone,gtwo,machine,human,label
0,MSFT_q4_2018.txt,And our commercial cloud business surpassed mo...,['cloud'],['Cloud computing'],Past,Past,0
1,MSFT_q4_2018.txt,I shared our vision for the intelligent cloud ...,['cloud'],['Cloud computing'],Past,Past,0
2,MSFT_q4_2018.txt,We reorganized our engineering teams to break ...,['AI'],['AI'],Past,Past,0
3,MSFT_q4_2018.txt,"We reoriented our sales and marketing teams, a...","['transformation', 'digital transformation']",['Practices'],Past,Past,0
4,MSFT_q4_2018.txt,"And, most importantly, we drove innovation to ...","['cloud', 'the cloud']",['Cloud computing'],Past,Past,0
...,...,...,...,...,...,...,...
396,IBM_q1_2018.txt,We’ve grown to over 50 active blockchain netwo...,"['platform', 'blockchain']","['Blockchain', 'Digital Product']",Pres,Pres,1
397,IBM_q1_2018.txt,"And last month, we announced a beta version of...","['platform', 'blockchain']","['Blockchain', 'Digital Product']",Past,Past,0
398,IBM_q1_2018.txt,Our strategic imperatives revenue reflects the...,['cloud'],['Cloud computing'],Pres,Pres,1
399,IBM_q1_2018.txt,Revenue growth was led by strong double-digit ...,"['transformation', 'digital transformation', '...","['Practices', 'Enablers']",Past,Past,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.25, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [ ]:
X_val

array([242, 214,  58, 300, 273,   7, 337, 234, 180, 146,  95, 235, 198,
       382,  97, 363,   1, 262, 113, 367, 173, 292, 323, 248, 326, 280,
        57, 134, 139, 112, 333, 116, 398, 334,  59, 176, 167,  67, 159,
        77, 299, 205,  37, 165,  29, 160, 394, 226,  94, 344, 396,  60,
       381, 339, 314, 389, 245, 357,  61, 128, 335, 137, 378, 105, 120,
        79, 210, 261, 233, 331, 296, 373, 174, 354,  22, 204,  85, 163,
       279, 127, 237,  84, 284, 241, 133, 399, 308,  38,  20, 229, 240,
       306, 254, 208, 211, 223,  75, 293, 386, 349, 191])

In [ ]:
y_train

array([0, 2, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 2, 1, 0,
       1, 1, 1, 0, 1, 2, 2, 1, 2, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 2, 1, 0, 1, 1, 0, 2, 1, 2, 0, 1, 1, 0, 0, 1, 0, 0, 0, 2, 0,
       2, 0, 1, 2, 0, 0, 2, 2, 0, 1, 0, 0, 2, 1, 0, 0, 2, 2, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 2, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 2, 2,
       1, 0, 2, 1, 0, 2, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 2, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 1, 2, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 2, 1,
       1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 2, 2, 1, 2, 1, 2, 0,
       1, 1, 0, 0, 2, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 0, 2, 2, 1, 1,
       0, 1, 0, 2, 1, 2, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 2, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 0, 1, 2, 1, 0, 2, 1, 0,
       0, 1, 0, 0, 2, 0, 0, 2, 1, 1, 0, 1, 1, 1])

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['label', 'data_type']).count()

id  text  gone  gtwo  machine  human
label data_type                                       
0     train      106   106   106   106      106    106
      val         36    36    36    36       36     36
1     train      147   147   147   147      144    147
      val         49    49    49    49       49     49
2     train       47    47    47    47       46     47
      val         16    16    16    16       16     16

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# Download configuration from huggingface.co and cache.
config = AutoConfig.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert",
                                                            num_labels=3,
                                                           output_attentions=False,
                                                           output_hidden_states=False)

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train), len(dataset_val)

(300, 101)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
epochs = 6

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    #label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        # print(f'Class: {label_dict_inverse[label]}')
        print(f'Class: {label}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

def predict(dataloader_test):

    model.eval()
    
    predictions = []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]
                  }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        # loss = outputs[0]
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
        
    predictions = np.concatenate(predictions, axis=0)            
    return predictions

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    # torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.2292323952540756
Validation loss: 0.8482163664610947
F1 Score (Weighted): 0.6126497265111127


Epoch 2:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.42901508502662183
Validation loss: 0.7667539550868028
F1 Score (Weighted): 0.76013755221676


...


Epoch 4
Training loss: 0.0839145962940529
Validation loss: 0.8087023945306154
F1 Score (Weighted): 0.8195656098439319


Epoch 5:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.034548215633258224
Validation loss: 0.8624161519110203
F1 Score (Weighted): 0.8195656098439319


Epoch 6:   0%|          | 0/100 [00:00<?, ?it/s]


Epoch 6
Training loss: 0.012573511465452611
Validation loss: 0.8828667219153002
F1 Score (Weighted): 0.8195656098439319


In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: 0
Accuracy: 32/36

Class: 1
Accuracy: 41/49

Class: 2
Accuracy: 10/16

